# Ponderada RAG

Essa ponderada tem como objetivo a implementação de um RAG, utilizando o framework Ollama para otimizar a recuperação e geração de respostas baseada em documentos. 

Para entendimento do Ollama, utilizou-se do seguinte tutorial: https://ollama.com/blog/embedding-models

## Instalação das Bibliotecas

- ollama
- numpy
- wikipedia-api
- scikit-learn

In [1]:
!pip install -q ollama numpy wikipedia-api scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Ollama Local

Para a ponderada, foi necessário rodar o framework do ollama de forma Local, seguindo o seguinte Passo a Passo;

1 - Download do Ollama: https://ollama.com/download

2 - No terminal:

```
brew install ollama
```

3 - Verificar se o ollama foi instalado

```
ollama --version
```

4 - Baixar os Modelos utilizados

```
ollama pull llama3
ollama pull mistral
```


## Importação das Bibliotecas

In [2]:
import ollama
import numpy as np
import wikipediaapi

from sklearn.metrics.pairwise import cosine_similarity

## Etapa 1: Retrieval

Para conseguir os documentos/informações, optei pela utilização da API do wikipedia. Por meio do uso da API, busquei detalhes sobre as seguintes linguagens de programacao:

- Rust: https://en.wikipedia.org/wiki/Rust_(programming_language)
- Go: https://en.wikipedia.org/wiki/Go_(programming_language)
- F*: https://en.wikipedia.org/wiki/F*_(programming_language)

Em seguida, cada pagina extraida do Wikipedia será transformado em um valor númerico em um processo conhecido como Embedding. Para o embedding, utilizou-se o llama3

Realizando o embedding nos documentos, conseguimos utilizar a similariedade do Cosseno para encontrar os arquivos mais relevantes.

#### Extração dos Documentos

In [ ]:
user_agent = "test@gmail.com"  # Substitua pelo seu e-mail

wikipedia_ingles = wikipediaapi.Wikipedia(
    user_agent=user_agent,
    language='en'
)

paginas = [
    "Rust (programming language)",
    "Go (programming language)",
    "F* (programming language)"
]

documentos = []

for pagina in paginas:
    documentos.append(wikipedia_ingles.page(pagina).summary)

for i in documentos:
    print(i, "\n" + "="*80 + "\n")

Rust is a general-purpose programming language emphasizing performance, type safety, and concurrency. It enforces memory safety, meaning that all references point to valid memory. It does so without a conventional garbage collector; instead, memory safety errors and data races are prevented by the "borrow checker", which tracks the object lifetime of references at compile time.
Rust does not enforce a programming paradigm, but was influenced by ideas from functional programming, including immutability, higher-order functions, algebraic data types, and pattern matching. It also supports object-oriented programming via structs, enums, traits, and methods. It is popular for systems programming.
Software developer Graydon Hoare created Rust as a personal project while working at Mozilla Research in 2006. Mozilla officially sponsored the project in 2009. In the years following the first stable release in May 2015, Rust was adopted by companies including Amazon, Discord, Dropbox, Google (Alp

#### Embedding dos Documentos

In [5]:
modelo_embedding = "llama3"

def gerar_embeddings(texto):
    resposta = ollama.embeddings(model=modelo_embedding, prompt=texto)
    return resposta["embedding"] 

embeddings = []

for x in documentos:
    embeddings.append(gerar_embeddings(x))

print(embeddings)

[[-4.54727029800415, -0.08652281761169434, -3.4683213233947754, 1.7166048288345337, 0.9376373291015625, 1.3093535900115967, -0.22836166620254517, -0.6303377747535706, -1.7315666675567627, -0.8980337381362915, -0.395516037940979, -0.5219791531562805, -2.5143299102783203, -3.6199934482574463, 0.8968530297279358, 1.082987904548645, -3.086148738861084, -2.6292543411254883, -1.446865439414978, -0.9386518597602844, 1.6123089790344238, -0.060048528015613556, -2.4179420471191406, -0.27638158202171326, 0.68658447265625, 0.09232531487941742, -1.9474189281463623, -5.70998477935791, -1.0796856880187988, 0.06239620968699455, -0.9225311279296875, 3.1114988327026367, -0.5633423924446106, 1.3464239835739136, 7.0665130615234375, -0.1869044452905655, 0.031820282340049744, 1.2279860973358154, 2.6361899375915527, 1.9089012145996094, -2.4334018230438232, -0.33456140756607056, 0.8514017462730408, -0.8391435146331787, -3.4114060401916504, 1.8603616952896118, 2.4533815383911133, 1.0688953399658203, 1.84604334

#### Arquivos Relevantes

In [11]:
def doc_relevantes(query, top_relevantes):
    query_embedding = [gerar_embeddings(query)]

    similaridade = cosine_similarity(query_embedding, embeddings)[0]
    pares_relevancia = list(enumerate(similaridade))

    def ordenar_por_similaridade(par):
        return par[1] 

    pares_ordenados = sorted(pares_relevancia, key=ordenar_por_similaridade, reverse=True)

    ranking_relevancia = []
    for par in pares_ordenados[:top_relevantes]: 
        ranking_relevancia.append(par[0])

    documentos_relevantes = []
    for indice in ranking_relevancia:
        documentos_relevantes.append(documentos[indice])

    return documentos_relevantes


query = "Which Software Company creates Go Language Programming"
top_relevantes = 3
docs_encontrados = doc_relevantes(query, top_relevantes)

print("Documentos relevantes:\n")
for i, doc in enumerate(docs_encontrados, start=1):
    print(f"{i}. {doc[:500]}...")
    print("=" * 80)

Documentos relevantes:

1. Go is a high-level general purpose programming language that is statically typed and compiled. It is known for the simplicity of its syntax and the efficiency of development that it enables by the inclusion of a large standard library supplying many needs for common projects. It was designed at Google in 2009 by Robert Griesemer, Rob Pike, and Ken Thompson. It is syntactically similar to C, but also has memory safety, garbage collection, structural typing, and CSP-style concurrency. It is often ...
2. Rust is a general-purpose programming language emphasizing performance, type safety, and concurrency. It enforces memory safety, meaning that all references point to valid memory. It does so without a conventional garbage collector; instead, memory safety errors and data races are prevented by the "borrow checker", which tracks the object lifetime of references at compile time.
Rust does not enforce a programming paradigm, but was influenced by ideas from functi

## Augmentation

Após a recuperação dos documentos mais relevantes, o próximo passo é otimizar o prompt para melhorar a geração da resposta final.

Para isso, iremos usar o modelo Mistral do Ollama pra refinar o prompt.

In [12]:
def refinar_prompt(pergunta, contexto, modelo_refinamento="mistral"):
    prompt_refinado = f"""
    Você é um especialista em engenharia de prompts. Seu objetivo é melhorar o seguinte prompt para obter uma resposta mais precisa:

    Contexto da pergunta:
    {contexto}

    Pergunta original:
    {pergunta}

    Gere um novo prompt estruturado, claro e otimizado para obter a melhor resposta possível. O novo prompt deve ser direto, sem repetições.
    """

    resposta_refinada = ollama.generate(model=modelo_refinamento, prompt=prompt_refinado)
    return resposta_refinada["response"]

## Generation

O resultado do Prompt Refinado é enviado para um 2 LLM (LLAMA3) para que ele consiga gerar a resposta final com base nas informações recuperadas

In [17]:
def gerar_resposta(pergunta, top_relevantes=3, modelo_refinamento="mistral", modelo_resposta="llama3"):
    paragrafos_relevantes = doc_relevantes(pergunta, top_relevantes=top_relevantes)
    contexto = "\n".join(paragrafos_relevantes)
    prompt_final = refinar_prompt(pergunta, contexto, modelo_refinamento)
    resposta = ollama.generate(model=modelo_resposta, prompt=prompt_final)

    return resposta["response"]

### Testes


In [20]:
pergunta = "Em que ano Rust foi criado?"
resposta = gerar_resposta(pergunta, top_relevantes=3)

print("\n💡 Resposta Gerada:")
print(resposta)


💡 Resposta Gerada:
A resposta correta é:

- O Rust foi criado em 2006 (por Graydon Hoare) enquanto ele trabalhava na Mozilla Research, embora o projeto tenha sido oficialmente financiado pela Mozilla em 2009.

O Rust foi fundado por Graydon Hoare em 2006, mas não alcançou uma versão estável até 2015.


In [21]:
pergunta = "Qual a diferença entre Go e F*??"
resposta = gerar_resposta(pergunta, top_relevantes=3)

print("\n💡 Resposta Gerada:")
print(resposta)


💡 Resposta Gerada:
Go (or Golang) and F* are two distinct programming languages with different design goals, features, and use cases. Here's a brief comparison:

**1. Purpose:**
	* Go (Golang): Designed for building scalable and concurrent systems, with a focus on simplicity, reliability, and efficiency. Ideal for developing cloud infrastructure, networks, and distributed systems.
	* F*: A functional programming language aimed at building reliable and maintainable software, particularly in the context of verification and formal methods.

**2. Typing:**
	* Go (Golang): Statically typed, which means that the compiler checks the types of variables at compile-time to prevent type errors at runtime.
	* F*: Also statically typed, with a focus on dependent types, which allow for expressing complex relationships between types and values.

**3. Concurrency:**
	* Go (Golang): Designed from the ground up with concurrency in mind, featuring lightweight goroutines, channels, and mutexes to handle 